In [ ]:
import numpy as np
import cv2 as cv
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
from matplotlib import pyplot as plt

## Preparing Dataset

In [ ]:
dataset_path = "7_7_dataset_faces"
width = height = 224
batch_size = 32

data_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),
    fill_mode='reflect',
    validation_split=0.1)

train_data = data_generator.flow_from_directory(
    dataset_path,
    target_size=(width, height),
    class_mode='categorical', 
    batch_size=batch_size,
    subset='training',
    shuffle=True
)

val_data = data_generator.flow_from_directory(
    dataset_path,
    target_size=(width, height),
    class_mode='categorical', 
    batch_size=batch_size,
    subset='validation',
    shuffle=True
)

In [ ]:
train_images = next(train_data)[0]
plt.figure(figsize=(8,8)) # plot 16 images

for i in range(16):
    plt.subplot(4,4,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)

## Create Model

Option 1

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(width, height, 3),
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling='avg'
)

In [ ]:
for layer in base_model.layers[:-4]:
    layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
model = tf.keras.Sequential([
    base_model,
    layers.Dropout(0.5),
    layers.Dense(train_data.num_classes, activation='softmax')
])

Option 2

In [ ]:
model = tf.keras.models.load_model('saved_model')

In [ ]:
model.summary()

## Train Model

In [ ]:
model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
    # optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    # optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.0001),
    metrics='accuracy'
)

In [ ]:
epochs = 32

history = model.fit(
    train_data,
    steps_per_epoch=train_data.samples // batch_size,
    validation_data=val_data,
    validation_steps=val_data.samples // batch_size,
    epochs=epochs,
    shuffle=True
)

In [ ]:
from matplotlib import pyplot as plt

plt.plot(history.history['loss'], label="Train")
plt.plot(history.history['val_loss'], label="Validation")
plt.legend(loc='best')
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.grid()
plt.show()

plt.plot(history.history['accuracy'], label="Train")
plt.plot(history.history['val_accuracy'], label="Validation")
plt.legend(loc='best')
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.grid()
plt.show()

## Test Model

In [ ]:
model.evaluate(val_data)

In [ ]:
import urllib.request

image_url = 'https://s.france24.com/media/display/a89f55a2-abff-11ea-b263-005056bff430/w:1280/p:16x9/2020-06-10T224534Z_26952549_RC2K6H9AXPGB_RTRMADP_3_MINNEAPOLIS-POLICE-TRUMP.webp'
urllib.request.urlretrieve(image_url, 'person.jpg')

In [ ]:
import cv2

img = cv2.imread("person.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224, 224))
img = img / 255.0
img = img.reshape(1, 224, 224, 3)

output = model.predict(img)
print(output)
print('probability', np.max(output))
print('class index', np.argmax(output))

labels = list(train_data.class_indices.keys())
print('class label', labels[np.argmax(output)])

In [ ]:
model.save('saved_model')